<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/concatenate_anndatas/concatenate_anndatas_half.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pre-processing of anndata

In [4]:
!pip install scikit-misc
from skmisc.loess import loess
!pip install scvi-tools==0.8.0b0


!pip install pillow scikit-learn



!pip install scanpy anndata

!pip install scanpy python-igraph louvain MulticoreTSNE pybiomart

#import packages
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import scvi


# Import packages
import anndata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.decomposition import TruncatedSVD
from scipy import sparse, io

matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

def nd(arr):
    return np.asarray(arr).reshape(-1)

     |████████████████████████████████| 10.3MB 6.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Created wheel for sinfo: filename=sinfo-0.3.1-cp36-none-any.whl size=7014 sha256=25bb0f94dcdec5127ebd07e3aab0ef9bb1c392d49b3955b3c778c7b25fd05d5e
  Stored in directory: /root/.cache/pip/wheels/11/f0/23/347d6d8e59787c2bc272162d18223dc3b45bd6dc40aceee6af
Successfully built sinfo
     |████████████████████████████████| 3.2MB 8.5MB/s 
     |████████████████████████████████| 2.2MB 65.6MB/s 
  Created wheel for MulticoreTSNE: filename=MulticoreTSNE-0.1-cp36-cp36m-linux_x86_64.whl size=68503 sha256=921d4f0e6f1bf251bc1e94d011b1586ef998b4b21a15d7490d650558f5034b82
  Stored in directory: /root/.cache/pip/wheels/27/59/53/3b52ee63add3692254c30d687fa4dff4d128d0557861fb028e
Successfully built MulticoreTSNE


We first need to transfer the metadata from the uns dictionary to obs columns. Will need a different notebook to do this but I can start drafting the code here.

## Download the data from Box

In [5]:
!wget -O results.zip https://caltech.box.com/shared/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m

--2021-02-04 17:17:41--  https://caltech.box.com/shared/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m
Resolving caltech.box.com (caltech.box.com)... 107.152.24.197
Connecting to caltech.box.com (caltech.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m [following]
--2021-02-04 17:17:41--  https://caltech.box.com/public/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m [following]
--2021-02-04 17:17:41--  https://caltech.app.box.com/public/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m
Resolving caltech.app.box.com (caltech.app.box.com)... 107.152.24.201
Connecting to caltech.app.box.com (caltech.app.box.com)|107.152.24.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: h

In [6]:
!unzip results.zip

Archive:  results.zip
   creating: results/
  inflating: __MACOSX/._results      
  inflating: results/resultGSM3319036.dms  
  inflating: __MACOSX/results/._resultGSM3319036.dms  
  inflating: results/resultGSM2803967.dms  
  inflating: __MACOSX/results/._resultGSM2803967.dms  
  inflating: results/resultGSM3509839  
  inflating: __MACOSX/results/._resultGSM3509839  
  inflating: results/resultGSM2905292.dms  
  inflating: __MACOSX/results/._resultGSM2905292.dms  
  inflating: results/resultGSM3439917  
  inflating: __MACOSX/results/._resultGSM3439917  
  inflating: results/resultGSM2905293.dms  
  inflating: __MACOSX/results/._resultGSM2905293.dms  
  inflating: results/resultGSM3439921  
  inflating: __MACOSX/results/._resultGSM3439921  
  inflating: results/resultGSM3439926  
  inflating: __MACOSX/results/._resultGSM3439926  
  inflating: results/resultGSM3439919  
  inflating: __MACOSX/results/._resultGSM3439919  
  inflating: results/resultGSM3319037.dms  
  inflating: __MACOSX/r

First get list of names of anndata objects

In [7]:
import os
adata_names = os.listdir("/content/results")

In [8]:
invalid_names = [name for name in adata_names if ("result" not in name and "SRR" not in name) or "fastq" in name]

In [9]:
invalid_names

['SRA_tables', 'SRR5481335_1.fastq.gz', '.DS_Store']

In [10]:
for invalid_name in invalid_names:
  adata_names.remove(invalid_name)

In [11]:
# Split the list in two and choose which half you want to run
  
#which_half = "first"
which_half = "second"

cutoff = len(adata_names)//2

if which_half == "first":
  adata_names = adata_names[:cutoff]

if which_half == "second":
  adata_names = adata_names[cutoff:]



In [12]:
def adata_to_dict(adata_names):
  '''
  Given a list of adata names, create dictionary 
  of adata objects using the sample ID
  of each adata object as key.
  Also, moves metadata from adata.uns to observation columns.
  '''
  adata_dict = {}
  for adata in adata_names:
    # Read adata into dictionary
    adata_dict[adata] = anndata.read_h5ad("/content/results/" + adata);

    adata_dict[adata].var_names_make_unique()

    # transfer metadata from uns to obs columns
    for key, value in adata_dict[adata].uns.items():
      adata_dict[adata].obs[key] = value

  return adata_dict


In [13]:
adata_dict = adata_to_dict(adata_names)


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Vari

In [14]:
#Get list of adata to concatenate
adata_list = [adata_dict[name] for name in adata_names]
adata_keys = [nd(adata_dict[name].obs["sample_id"])[0] for name in adata_names]



In [15]:
# Turns out we have duplicated names from GSE114530 (did with SRA table) Until I figure this out I will just rename then:

def uniq(input):
  a = 0
  output = []
  for x in input:
    if x not in output:
      output.append(x)
    else:
      output.append(x + str(a))
      a+=1
  return output


adata_keys_unique = uniq(adata_keys)


In [16]:
adata_half = anndata.concat(adata_list, join='outer', index_unique="_", label = "sample_id", keys = adata_keys_unique)

In [17]:
del adata_list
del adata_dict


In [19]:
if which_half == "first":
  adata_half.write("first_half_adata.dms")

if which_half == "second":
  adata_half.write("second_half_adata.dms")

... storing 'cell_type' as categorical
... storing 'condition' as categorical
... storing 'database_id' as categorical
... storing 'figure' as categorical
... storing 'paper' as categorical
... storing 'species' as categorical
... storing 'technology' as categorical
... storing 'tissue' as categorical


In [20]:
!gzip /content/second_half_adata.dms 

In [ ]:
import time
time.sleep(60000)

# Concatenate adatas (ended up doing this in the cluster)

Only run after having created and downloaded both halfs. If so, start running from here.

In [1]:
!pip install scikit-misc
from skmisc.loess import loess
!pip install scvi-tools==0.8.0b0


!pip install pillow scikit-learn



!pip install scanpy anndata

!pip install scanpy python-igraph louvain MulticoreTSNE pybiomart

#import packages
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import scvi


# Import packages
import anndata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.decomposition import TruncatedSVD
from scipy import sparse, io

matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

def nd(arr):
    return np.asarray(arr).reshape(-1)

### Download the two halves from Box

In [2]:
!wget -O first_half.dms https://caltech.box.com/shared/static/9j4eieuydrky2bf091o9kqsqzm0652by.dms

!wget -O second_half.dms.gz https://caltech.box.com/shared/static/jxj7p5d8w40yw6b5pbdvhor76qo6mufo.gz

--2021-02-04 22:04:00--  https://caltech.box.com/shared/static/9j4eieuydrky2bf091o9kqsqzm0652by.dms
Resolving caltech.box.com (caltech.box.com)... 185.235.236.197
Connecting to caltech.box.com (caltech.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9j4eieuydrky2bf091o9kqsqzm0652by.dms [following]
--2021-02-04 22:04:00--  https://caltech.box.com/public/static/9j4eieuydrky2bf091o9kqsqzm0652by.dms
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/9j4eieuydrky2bf091o9kqsqzm0652by.dms [following]
--2021-02-04 22:04:00--  https://caltech.app.box.com/public/static/9j4eieuydrky2bf091o9kqsqzm0652by.dms
Resolving caltech.app.box.com (caltech.app.box.com)... 185.235.236.201
Connecting to caltech.app.box.com (caltech.app.box.com)|185.235.236.201|:443... connected.
HTTP request sent, awaiting response.

In [3]:
!gunzip second_half.dms.gz

gzip: second_half.dms already exists; do you wish to overwrite (y or n)? ^C


Concatenate them

In [4]:
#read
adata_first_half = anndata.read_h5ad("first_half.dms")
adata_second_half = anndata.read_h5ad("second_half.dms")


In [ ]:
adata = anndata.concat([adata_first_half, adata_second_half], join='outer', index_unique= None)

In [ ]:
adata.write("adata_concatenated.dms")

In [ ]:
import time
time.sleep(60000)